In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, concatenate, BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, SeparableConv2D, DepthwiseConv2D, AveragePooling2D

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip "/content/drive/My Drive/MRNet-v1.0"

In [ ]:
def read_data(data_type, scan_type):
  list = []
  for filename in sorted(os.listdir("/content/MRNet-v1.0/"+data_type+"/"+scan_type)):
    if(filename != ".DS_Store"):
      instance = np.load("/content/MRNet-v1.0/"+data_type+"/"+scan_type+"/" + filename)
      mid = len(instance) // 2
      instance = np.array([ instance[mid-1] , instance[mid] ,instance[mid+1]])
      list.append(instance)
  data = np.asarray(list)
  data = np.moveaxis(data, 1, 3)
  print(scan_type+data_type, data.shape)
  return data

def read_labels(data_type, scan_type):
  path = "/content/MRNet-v1.0/"+data_type+"-"+scan_type + ".csv"
  df=pd.read_csv(path, sep=',',header=None)
  labels = df.values
  labels = labels[:,1]
  print(scan_type+data_type, labels.shape)
  return labels


def read_data_singles(data_type, scan_type, label_scan_type):
  list = []
  path = "/content/MRNet-v1.0/"+data_type+"-"+label_scan_type + ".csv"
  df=pd.read_csv(path, sep=',',header=None)
  labels = df.values
  labels = labels[:,1]
  single_labels = []
  i=0
  for filename in sorted(os.listdir("/content/MRNet-v1.0/"+data_type+"/"+scan_type)):
    if(filename != ".DS_Store"):
      instance = np.load("/content/MRNet-v1.0/"+data_type+"/"+scan_type+"/" + filename)
      for image in instance:
        list.append(np.array(image))
        single_labels.append(labels[i])
      i += 1
  data = np.asarray(list)
  data = data[:, :, :, np.newaxis]
  single_labels = np.asarray(single_labels)
  print("single")
  print(scan_type+data_type, data.shape)
  print(label_scan_type+data_type, single_labels.shape)
  return data, single_labels

single_sagittal_train_x, single_saggital_abnormal_train_y = read_data_singles("train","sagittal","abnormal")
single_sagittal_test_x, single_saggital_abnormal_test_y = read_data_singles("valid","sagittal","abnormal")

# print("Data: ")

# sagittal_train_x = read_data("train","sagittal")
# axial_train_x = read_data("train","axial")
# coronal_train_x = read_data("train","coronal")

# sagittal_test_x = read_data("valid","sagittal")
# axial_test_x = read_data("valid","axial")
# coronal_test_x = read_data("valid","coronal")

# print("Labels: ")
# abnormal_train_y = read_labels('train','abnormal')
# acl_train_y = read_labels('train','acl')
# meniscus_train_y = read_labels('train','meniscus')

# abnornmal_test_y = read_labels('valid','abnormal')
# acl_test_y = read_labels('valid','acl')
# meniscus_test_y = read_labels('valid','meniscus')

single
sagittaltrain (34370, 256, 256, 1)
abnormaltrain (34370,)
single
sagittalvalid (3663, 256, 256, 1)
abnormalvalid (3663,)


In [ ]:
def inception_block_A(inputs):
  branch_1 = conv2d_bn(inputs, kernel_size=(1,1), filters=64, strides = 1, padding="same")
  branch_1 = conv2d_bn(branch_1,kernel_size=(3,3), filters=96, strides = 1, padding="same")
  branch_1 = conv2d_bn(branch_1, kernel_size=(3,3), filters=96, strides = 1, padding="same")

  branch_2 = conv2d_bn(inputs, kernel_size=(1,1), filters=64, strides = 1, padding="same")
  branch_2 = conv2d_bn(branch_2, kernel_size=(3,3), filters=96, strides = 1, padding="same")

  branch_3 = conv2d_bn(inputs, kernel_size=(1,1), filters=96, strides = 1, padding="same")

  branch_4 = AveragePooling2D((3,3), strides = 1, padding="same")(inputs)
  branch_4 = conv2d_bn(branch_4, kernel_size=(1,1), filters=96, strides = 1, padding="same")
  output   =  concatenate([branch_1, branch_2, branch_3, branch_4], axis=-1)

  return output

def inception_block_B(inputs):
  branch_1 = conv2d_bn(inputs, kernel_size=(1,1), filters=192, strides = 1, padding="same")
  branch_1 = conv2d_bn(branch_1, kernel_size=(1,7), filters=192, strides = 1, padding="same")
  branch_1 = conv2d_bn(branch_1, kernel_size=(7,1), filters=224, strides = 1, padding="same")
  branch_1 = conv2d_bn(branch_1, kernel_size=(1,7), filters=224, strides = 1, padding="same")
  branch_1 = conv2d_bn(branch_1, kernel_size=(7,1), filters=256, strides = 1, padding="same")


  branch_2 = conv2d_bn(inputs, kernel_size=(1,1), filters=192, strides = 1, padding="same")
  branch_2 = conv2d_bn(branch_2, kernel_size=(1,7), filters=224, strides = 1, padding="same")
  branch_2 = conv2d_bn(branch_2, kernel_size=(1,7), filters=256, strides = 1, padding="same")


  branch_3 = conv2d_bn(inputs, kernel_size=(1,1), filters=384, strides = 1, padding="same")

  branch_4 = AveragePooling2D((3,3), strides = 1, padding="same")(inputs)
  branch_4 = conv2d_bn(branch_4, kernel_size=(1,1), filters=128, strides = 1, padding="same")
  output =  concatenate([branch_1, branch_2, branch_3, branch_4], axis=-1)

  return output

def inception_block_C(inputs):
  branch_1 = conv2d_bn(inputs, kernel_size=(1,1), filters=384, strides = 1, padding="same")
  branch_1 = conv2d_bn(branch_1, kernel_size=(1,3), filters=448, strides = 1, padding="same")
  branch_1 = conv2d_bn(branch_1, kernel_size=(3,1), filters=512, strides = 1, padding="same")
  branch_1_1 = conv2d_bn(branch_1, kernel_size=(1,3), filters=256, strides = 1, padding="same")
  branch_1_2 = conv2d_bn(branch_1, kernel_size=(3,1), filters=256, strides = 1, padding="same")

  branch_2 = conv2d_bn(inputs, kernel_size=(1,1), filters=384, strides = 1, padding="same")
  branch_2_1 = conv2d_bn(branch_2, kernel_size=(3,1), filters=256, strides = 1, padding="same")
  branch_2_2 = conv2d_bn(branch_2, kernel_size=(1,3), filters=256, strides = 1, padding="same")

  branch_3 = conv2d_bn(inputs, kernel_size=(1,1), filters=256, strides = 1, padding="same")

  branch_4 = AveragePooling2D((3,3), strides = 1, padding="same")(inputs)
  branch_4 = conv2d_bn(branch_4, kernel_size=(1,1), filters=256, strides = 1, padding="same")
  output =  concatenate([branch_1_1, branch_1_2, branch_2_1, branch_2_2 , branch_3, branch_4], axis=-1)

  return output

def reduction_block_A(inputs):
  branch_1 = conv2d_bn(inputs, kernel_size=(1,1), filters=192, strides = 1, padding="same")
  branch_1 = conv2d_bn(branch_1, kernel_size=(3,3), filters=224, strides = 1, padding="same")
  branch_1 = conv2d_bn(branch_1, kernel_size=(3,3), filters=256, strides = 2, padding="valid")
   
  branch_2 = conv2d_bn(inputs, kernel_size=(3,3), filters=384, strides = 2, padding="valid")

  branch_3 = AveragePooling2D((3,3), strides = 2)(inputs)
  output =  concatenate([branch_1, branch_2, branch_3], axis=-1)

  return output

def reduction_block_B(inputs):
  branch_1 = conv2d_bn(inputs, kernel_size=(1,1), filters=256, strides = 1, padding="same")
  branch_1 = conv2d_bn(branch_1, kernel_size=(1,7), filters=256, strides = 1, padding="same")
  branch_1 = conv2d_bn(branch_1, kernel_size=(7,1), filters=320, strides = 1, padding="same")
  branch_1 = conv2d_bn(branch_1, kernel_size=(3,3), filters=320, strides = 2, padding="valid")

  branch_2 = conv2d_bn(inputs, kernel_size=(1,1), filters=192, strides = 1, padding="same")
  branch_2 = conv2d_bn(branch_2, kernel_size=(3,3), filters=192, strides = 2, padding="valid")

  branch_3 = AveragePooling2D((3,3), strides = 2)(inputs)

  output =  concatenate([branch_1, branch_2, branch_3], axis=-1)

  return output


def inception_block(inception_type, inputs):
  if inception_type == "A":
    return inception_block_A(inputs)
  if inception_type == "B":
    return inception_block_B(inputs)
  if inception_type == "C":
    return inception_block_C(inputs)

def conv2d_bn(inputs ,filters, kernel_size, padding, strides):
  inputs = Conv2D(filters, kernel_size, strides=strides, padding=padding)(inputs)
  inputs = BatchNormalization(axis = 3)(inputs)
  outputs = Activation('relu')(inputs)
  return outputs

In [ ]:
inputs = Input(shape=(256, 256, 1))  

x = conv2d_bn(inputs, kernel_size=(3,3), filters=32, strides=2, padding="valid")
x = conv2d_bn(x, kernel_size=(3,3), filters=32, strides=1, padding="valid")
x = conv2d_bn(x, kernel_size=(3,3), filters=64, strides=1, padding="same")
x_1 = MaxPooling2D((3,3), strides = 2)(x)
x_2 = conv2d_bn(x, kernel_size=(3,3), filters=96, strides=2, padding="valid")
x = concatenate([x_1, x_2], axis=-1)
x_1 = conv2d_bn(x, kernel_size=(1,1), filters=64, strides=1, padding="same")
x_1 = conv2d_bn(x_1, kernel_size=(3,3), filters=96, strides=1, padding="valid")
x_2 = conv2d_bn(x, kernel_size=(1,1), filters=64, strides=1, padding="same")
x_2 = conv2d_bn(x_2, kernel_size=(7,1), filters=64, strides=1, padding="same")
x_2 = conv2d_bn(x_2, kernel_size=(1,7), filters=64, strides=1, padding="same")
x_2 = conv2d_bn(x_2, kernel_size=(3,3), filters=96, strides=1, padding="valid")
x = concatenate([x_1, x_2], axis=-1)
x_1 = conv2d_bn(x, kernel_size=(3,3), filters=192, strides=2, padding="valid")
x_2 = MaxPooling2D((3,3), strides = 2)(x)
x = concatenate([x_1, x_2], axis=-1)

inception_A  = inception_block("A", x)
for i in range(3): inception_A  = inception_block("A", inception_A)

reduction_A  = reduction_block_A(inception_A)

inception_B  = inception_block("B", reduction_A)
for i in range(6):inception_B  = inception_block("B", inception_B)

reduction_B  = reduction_block_B(inception_B)

inception_C  = inception_block("C", reduction_B)
inception_C = inception_block("C", inception_C)

global_pool = GlobalAveragePooling2D()(inception_C)
dense = Flatten()(global_pool)
dense = Dense(1000, activation='relu')(dense)
dense = Dropout(0.7)(dense)
output = Dense(1, activation='sigmoid')(dense)

model = Model(inputs, output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 127, 127, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 127, 127, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 127, 127, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.fit(single_sagittal_train_x, single_saggital_abnormal_train_y, batch_size=32, epochs=15, validation_split=0.1, verbose=1, shuffle=True)

Epoch 1/15
967/967 [==============================] - 369s 382ms/step - loss: 0.4019 - accuracy: 0.8350 - val_loss: 0.3856 - val_accuracy: 0.8385
Epoch 2/15
967/967 [==============================] - 366s 378ms/step - loss: 0.3642 - accuracy: 0.8438 - val_loss: 0.3967 - val_accuracy: 0.8280
Epoch 3/15
967/967 [==============================] - 364s 376ms/step - loss: 0.3459 - accuracy: 0.8487 - val_loss: 0.3727 - val_accuracy: 0.8344
Epoch 4/15
967/967 [==============================] - 362s 375ms/step - loss: 0.3284 - accuracy: 0.8556 - val_loss: 0.4151 - val_accuracy: 0.8315
Epoch 5/15
967/967 [==============================] - 363s 375ms/step - loss: 0.3131 - accuracy: 0.8615 - val_loss: 0.3750 - val_accuracy: 0.8362
Epoch 6/15
967/967 [==============================] - 361s 374ms/step - loss: 0.2933 - accuracy: 0.8703 - val_loss: 0.4114 - val_accuracy: 0.8257
Epoch 7/15
967/967 [==============================] - 362s 374ms/step - loss: 0.2763 - accuracy: 0.8788 - val_loss: 0.4543 -

In [ ]:
model.evaluate(single_sagittal_test_x, single_saggital_abnormal_test_y, batch_size=32)

115/115 [==============================] - 13s 109ms/step - loss: 0.5510 - accuracy: 0.8220


[0.5509756803512573, 0.8220038414001465]